We consider a multi-resolution diffusion process where diffusion proceeds at a given resolution for $T_\ell$ steps, then the variable is downsampled to produce the initial state of the next level:


$$
x_0^0 \to x_1^0 \to \cdots \to x_{T_0}^0 \;
\xrightarrow{\text{downsample}}\; x_0^1 \to x_1^1 \to \cdots \to x_{T_1}^1 \;
\xrightarrow{\text{downsample}}\; x_0^2 \to \cdots.
$$


Each **level** $\ell$ has its own spatial resolution and its own noise schedule $\{\beta_t^\ell\}_{t=1}^{T_\ell}$.
Let $D_\ell$ denote the fixed downsampling operator used to map $x_{T_\ell}^\ell$ into $x_0^{\ell+1}$.


We assume each level uses the standard DDPM forward dynamics:


$$
x_t^\ell = \sqrt{\alpha_t^\ell} \; x_{t-1}^\ell + \sqrt{1-\alpha_t^\ell} \; \varepsilon_t,
\qquad \varepsilon_t \sim \mathcal{N}(0, I).
$$


where $\alpha_t^\ell = 1 - \beta_t^\ell$ and $\bar\alpha_t^\ell = \prod_{s=1}^t \alpha_s^\ell$.


Our goal is to derive:
1. The **marginal** distribution $q(x_t^\ell \mid x_0^\ell)$.
2. The distribution of $x_0^{\ell+1}$ given $x_0^\ell$.
3. A minimal PyTorch implementation.

- $d_\ell$ denotes the dimension at level $\ell$ (e.g. number of pixels * channels at that downsampled resolution).
- All within-level transitions are linear-Gaussian and independent across injected noises.
- $D_\ell$ are linear downsampling operators (for $\ell=1$ this maps from level 0 to level 1). Define $D_0 = I$ for convenience.
- NOTE - for now downsamples should be linear

**Claim.** The marginal of $x_t^{(\ell)}$ given the original fine $x_0^{(0)}$ is Gaussian,


$$
q\big(x_t^{(\ell)} \mid x_0^{(0)}\big) = \mathcal{N}\big(x_t^{(\ell)};\; \mu_t^{(\ell)},\; \Sigma_t^{(\ell)}\big),
$$


with


$$
\mu_t^{(\ell)} = \sqrt{\bar\alpha_t^{(\ell)}}\, D_\ell\, \sqrt{\bar\alpha_{T_{\ell-1}}^{(\ell-1)}}\, D_{\ell-1}\, \cdots\, \sqrt{\bar\alpha_{T_0}^{(0)}}\, x_0^{(0)}.
$$


and the recursive covariance


$$
(1 - \bar\alpha_t^{(\ell)})\, I_{d_\ell}
   + \bar\alpha_t^{(\ell)}\!\left(
        D_\ell\, \Sigma_{T_{\ell-1}}^{(\ell-1)}\, D_\ell^\top 
        + \Sigma^{\mathrm{down}}_\ell
     \right).
$$


(Interpret $\Sigma_{T_{-1}}^{(-1)} = 0$ and $D_0 = I$.)




**Derivation.** Start from the within-level decomposition


$$
x_t^{(\ell)} = \sqrt{\bar\alpha_t^{(\ell)}} x_0^{(\ell)} + \varepsilon_t^{(\ell)}, \qquad \varepsilon_t^{(\ell)}\sim\mathcal{N}(0,(1-\bar\alpha_t^{(\ell)})I).
$$


Write the level-0 initialization of this level as


$$
x_0^{(\ell)} = D_\ell x_{T_{\ell-1}}^{(\ell-1)} + \zeta_\ell, \qquad \zeta_\ell\sim\mathcal{N}(0,\Sigma^{\mathrm{down}}_\ell).
$$


Substitute to get


$$
x_t^{(\ell)} = \sqrt{\bar\alpha_t^{(\ell)}} D_\ell x_{T_{\ell-1}}^{(\ell-1)} + \sqrt{\bar\alpha_t^{(\ell)}} \zeta_\ell + \varepsilon_t^{(\ell)}.
$$


Conditioning on $x_0^{(0)}$ we know $x_{T_{\ell-1}}^{(\ell-1)}$ has marginal mean $\mu_{T_{\ell-1}}^{(\ell-1)}$ and covariance $\Sigma_{T_{\ell-1}}^{(\ell-1)}$. Thus the mean is


$$
\mu_t^{(\ell)} = \sqrt{\bar\alpha_t^{(\ell)}} D_\ell \mu_{T_{\ell-1}}^{(\ell-1)}
$$


and the covariance is the sum of independent contributions:

$$
\begin{aligned}
\Sigma_t^{(\ell)} 
    &= \operatorname{Cov}\big(\sqrt{\bar\alpha_t^{(\ell)}} D_\ell x_{T_{\ell-1}}^{(\ell-1)}\big) + \operatorname{Cov}\big(\sqrt{\bar\alpha_t^{(\ell)}} \zeta_\ell\big) + \operatorname{Cov}(\varepsilon_t^{(\ell)}) \\
    &= \bar\alpha_t^{(\ell)} D_\ell \Sigma_{T_{\ell-1}}^{(\ell-1)} D_\ell^\top 
        + \bar\alpha_t^{(\ell)} \Sigma^{\mathrm{down}}_\ell 
        +  (1-\bar\alpha_t^{(\ell)}) I_{d_\ell} \\
&= (1 - \bar\alpha_t^{(\ell)})\, I_{d_\ell}
   + \bar\alpha_t^{(\ell)}\!\left(
        D_\ell\, \Sigma_{T_{\ell-1}}^{(\ell-1)}\, D_\ell^\top 
        + \Sigma^{\mathrm{down}}_\ell
     \right).
\end{aligned}
$$


This recursive formula allows computing marginals level-by-level.

In [1]:
import torch
import torch.nn.functional as F
from typing import List, Tuple, Optional

In [2]:
def make_linear_beta_schedule(T: int, beta_start: float = 1e-4, beta_end: float = 0.02) -> torch.Tensor:
    """Return beta_1..beta_T (length-T tensor)."""
    return torch.linspace(beta_start, beta_end, T)

In [3]:
betas = make_linear_beta_schedule(1000)

In [4]:
def alphas_bars_from_betas(betas: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    alphas = 1.0 - betas
    alphas_bar = torch.cumprod(alphas, dim=0)
    return alphas, alphas_bar

In [5]:
alphas, alphas_bar = alphas_bars_from_betas(betas)

In [6]:
def flatten_batch(x: torch.Tensor) -> torch.Tensor:
    return x.view(x.size(0), -1)

def unflatten(flat: torch.Tensor, sample_shape: Tuple[int, int, int]) -> torch.Tensor:
    return flat.view(-1, *sample_shape)

In [7]:
import torch

class AvgPoolDownsample:
    def __init__(self, kernel: int = 2, stride: Optional[int] = None):
        self.kernel = kernel
        self.stride = kernel if stride is None else stride

    def D(self, x: torch.Tensor) -> torch.Tensor:
        return torch.nn.functional.avg_pool2d(x, kernel_size=self.kernel, stride=self.stride)

    def DT(self, y: torch.Tensor) -> torch.Tensor:
        k = self.kernel
        y_expanded = y.repeat_interleave(k, dim=2).repeat_interleave(k, dim=3)
        return y_expanded / (k * k)

    def mat(self, in_shape: Tuple[int,int,int]) -> torch.Tensor:
        """
        Returns sparse COO matrix implementing avg-pool.
        Shape: (d_out, d_in)
        """
        C, H, W = in_shape
        k = self.kernel
        s = self.stride

        if k != s:
            raise NotImplementedError("Sparse matrix coded for non-overlapping pooling (kernel=stride).")

        if H % k != 0 or W % k != 0:
            raise ValueError("H and W must be divisible by kernel for kernel=stride avg-pool.")

        H2 = H // k
        W2 = W // k

        d_in = C * H * W
        d_out = C * H2 * W2

        rows = []
        cols = []
        vals = []

        # Index helpers
        def in_index(c, h, w):
            return c * (H * W) + h * W + w

        def out_index(c, h2, w2):
            return c * (H2 * W2) + h2 * W2 + w2

        # Build sparse entries
        for c in range(C):
            for y2 in range(H2):
                for x2 in range(W2):
                    out_i = out_index(c, y2, x2)
                    h0 = y2 * k
                    w0 = x2 * k

                    for i in range(k):
                        for j in range(k):
                            h = h0 + i
                            w = w0 + j
                            rows.append(out_i)
                            cols.append(in_index(c, h, w))
                            vals.append(1.0 / (k * k))

        indices = torch.tensor([rows, cols], dtype=torch.long)
        values = torch.tensor(vals, dtype=torch.float32)
        D = torch.sparse_coo_tensor(indices, values, (d_out, d_in))

        return D.coalesce()

In [8]:
down = AvgPoolDownsample(kernel=2)
D = down.mat((1,4,4))

x = torch.arange(16, dtype=torch.float32).view(1,1,4,4)
xf = x.view(-1)

y_sparse = (D @ xf)
y_true = torch.nn.functional.avg_pool2d(x, 2).view(-1)

assert torch.allclose(y_sparse, y_true)

In [9]:
D.shape

torch.Size([4, 16])

In [10]:
import torch

@torch.no_grad()
def precompute_t0_marginals_sparse(
    downsamplers: list,
    downsample_sigmas: list,
    level_shapes: list
):
    """
    Precompute t=0 marginals (mean and covariance) for each level.

    Parameters
    ----------
    downsamplers : list of AvgPoolDownsample
        Each downsampler maps level l-1 -> l
    downsample_sigmas : list of torch.Tensor or None
        Covariance for downsample noise per level
    level_shapes : list of tuples
        [(C,H0,W0), (C,H1,W1), ...] include level 0

    Returns
    -------
    marginals : list of dicts per level with keys:
        - 'mu0': sparse or dense linear map d_l x d0 (to multiply x0)
        - 'Sigma0': dense covariance d_l x d_l
        - 'd_out': output dimension
    """
    L = len(level_shapes) - 1  # number of downsample steps
    marginals = []

    # Level 0: identity
    C0,H0,W0 = level_shapes[0]
    d0 = C0*H0*W0
    marginals.append(dict(
        mu0=torch.eye(d0),    # d0 x d0
        Sigma0=torch.zeros((d0,d0)),
        d_out=d0
    ))

    for l in range(1,L+1):
        # previous level t0
        prev = marginals[l-1]
        mu_prev = prev['mu0']      # d_{l-1} x d0
        Sigma_prev = prev['Sigma0']  # d_{l-1} x d_{l-1}

        # downsample
        D_l = downsamplers[l-1].mat(level_shapes[l-1])  # d_l x d_{l-1}, sparse
        d_l = D_l.shape[0]

        # propagate mean: mu0^{(l)} = D_l @ mu_prev
        mu0_l = D_l @ mu_prev

        # propagate covariance: Sigma0^{(l)} = D_l Sigma_prev D_l^T + Sigma_down
        if downsample_sigmas is not None and downsample_sigmas[l-1] is not None:
            Sigma_down = downsample_sigmas[l-1]
        else:
            Sigma_down = torch.zeros((d_l,d_l))
        Sigma0_l = D_l @ Sigma_prev @ D_l.T + Sigma_down

        marginals.append(dict(
            mu0=mu0_l,
            Sigma0=Sigma0_l,
            d_out=d_l
        ))

    return marginals


In [11]:
# Example: 3 levels, 8x8 -> 4x4 -> 2x2, C=1
level_shapes = [(1,8,8), (1,4,4), (1,2,2)]
downsamplers = [AvgPoolDownsample(2), AvgPoolDownsample(2)]
downsample_sigmas = [None, None]  # deterministic

marginals_t0 = precompute_t0_marginals_sparse(downsamplers, downsample_sigmas, level_shapes)

# Level 0
print("Level 0:", marginals_t0[0]['mu0'].shape, marginals_t0[0]['Sigma0'].shape)
# Level 1
print("Level 1:", marginals_t0[1]['mu0'].shape, marginals_t0[1]['Sigma0'].shape)
# Level 2
print("Level 2:", marginals_t0[2]['mu0'].shape, marginals_t0[2]['Sigma0'].shape)


Level 0: torch.Size([64, 64]) torch.Size([64, 64])
Level 1: torch.Size([16, 64]) torch.Size([16, 16])
Level 2: torch.Size([4, 64]) torch.Size([4, 4])


In [12]:
marginals_t0[0]

{'mu0': tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]]),
 'Sigma0': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 'd_out': 64}

In [13]:
def avgpool_downsample_sigma(d_in: int, kernel_size: int, sigma_in: float) -> torch.Tensor:
    """
    Compute downsample noise covariance for average pooling.
    
    Parameters
    ----------
    d_in : int
        Input dimension (flattened)
    kernel_size : int
        Pooling kernel (assumes square kernel)
    sigma_in : float
        Original isotropic input variance
    
    Returns
    -------
    Sigma_down : torch.Tensor
        Dense covariance d_out x d_out for injection after pooling
    """
    factor = 1.0 / kernel_size**2
    sigma_out = sigma_in * factor
    sigma_extra = sigma_in - sigma_out
    d_out = d_in // (kernel_size**2)
    return sigma_extra * torch.eye(d_out)


In [14]:
import torch
from typing import Optional, List

@torch.no_grad()
def precompute_t0_marginals_sparse_full(
    downsamplers: List['AvgPoolDownsample'],
    downsample_sigmas: Optional[List[torch.Tensor]],
    level_shapes: List[tuple],
    betas_per_level: List[torch.Tensor],
    device: Optional[torch.device] = None
):
    """
    Precompute t=0 marginals (mean linear map and covariance) after downsampling,
    taking into account the last DDPM marginal of the previous level. If 
    downsample_sigmas[l] is None, computes exact covariance for average pooling.
    
    Parameters
    ----------
    downsamplers : list of AvgPoolDownsample
        Each downsampler maps level l-1 -> l
    downsample_sigmas : list of torch.Tensor or None
        Covariance for downsample noise per level
    level_shapes : list of tuples
        [(C,H0,W0), (C,H1,W1), ...] include level 0
    betas_per_level : list of torch.Tensor
        DDPM betas per level (length T_l)
    device : torch.device
    
    Returns
    -------
    marginals : list of dicts per level with keys:
        - 'mu0': sparse linear map d_l x d0 (multiply by x0_flat)
        - 'Sigma0': dense covariance d_l x d_l
        - 'd_out': output dimension
    """
    L = len(level_shapes) - 1  # number of downsample steps
    marginals = []

    # Level 0: identity
    C0,H0,W0 = level_shapes[0]
    d0 = C0*H0*W0
    I0 = torch.eye(d0, device=device)
    marginals.append(dict(
        mu0=I0.to_sparse(),
        Sigma0=torch.zeros((d0,d0), device=device),
        d_out=d0
    ))

    for l in range(1, L+1):
        prev = marginals[l-1]
        mu0_prev = prev['mu0']       # sparse d_{l-1} x d0
        Sigma0_prev = prev['Sigma0'] # dense d_{l-1} x d_{l-1}

        # Previous level final marginal
        betas_prev = betas_per_level[l-1].to(device)
        alphas_prev = 1 - betas_prev
        alphas_bar_prev = torch.cumprod(alphas_prev, dim=0)
        alpha_bar_T_prev = alphas_bar_prev[-1] if alphas_bar_prev.numel() > 0 else 1.0

        # Scale mean and covariance to previous level T
        mu_T_prev = mu0_prev.clone()
        if l-1 > 0:
            mu_T_prev = torch.sqrt(alpha_bar_T_prev) * mu0_prev
        Sigma_T_prev = (1 - alpha_bar_T_prev) * torch.eye(Sigma0_prev.shape[0], device=device) \
                       + alpha_bar_T_prev * Sigma0_prev

        # downsample sparse matrix
        D_l = downsamplers[l-1].mat(level_shapes[l-1])  # sparse d_l x d_{l-1}
        d_l = D_l.shape[0]

        # propagate mean and covariance through downsample
        mu0_l = torch.sparse.mm(D_l, mu_T_prev)

        # compute downsample noise if not given
        if downsample_sigmas is None or downsample_sigmas[l-1] is None:
            # assume AvgPoolDownsample
            pool_kernel = downsamplers[l-1].kernel
            # variance factor per output: sigma_out^2 = sigma_in^2 / k^2
            sigma_in2 = torch.mean(torch.diag(Sigma_T_prev)) if Sigma_T_prev.numel() > 0 else 1.0
            sigma_extra = sigma_in2 * (1 - 1.0 / pool_kernel**2)
            Sigma_down = sigma_extra * torch.eye(d_l, device=device)
        else:
            Sigma_down = downsample_sigmas[l-1].to(device)

        Sigma0_l = (D_l @ Sigma_T_prev.to_dense()) @ D_l.T.to_dense() + Sigma_down

        marginals.append(dict(
            mu0=mu0_l.coalesce(),
            Sigma0=Sigma0_l,
            d_out=d_l
        ))

    return marginals


In [15]:
def get_marginal(l: int, t: int, x0: torch.Tensor,
                 marginals_t0: list,
                 betas_per_level: list) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Return marginal mean and covariance of x_t^{(l)} given x0.

    Parameters
    ----------
    l : int
        Level index (0..L)
    t : int
        Time index within level l (0..T_l-1)
    x0 : torch.Tensor
        Flattened x0 of shape (d0, 1)
    marginals_t0 : list
        Precomputed t=0 marginals (output of precompute_t0_marginals_sparse_full)
    betas_per_level : list of torch.Tensor
        DDPM betas per level

    Returns
    -------
    mu_t : torch.Tensor
        Marginal mean at level l, shape (d_l, 1)
    Sigma_t : torch.Tensor
        Marginal covariance at level l, shape (d_l, d_l)
    """
    # t=0 marginal for this level
    m0 = marginals_t0[l]
    mu0 = torch.sparse.mm(m0['mu0'], x0) if l > 0 else x0  # shape (d_l,1)
    Sigma0 = m0['Sigma0']  # shape (d_l,d_l)

    # within-level DDPM scaling
    betas = betas_per_level[l].to(x0.device)
    alphas = 1 - betas
    alphas_bar = torch.cumprod(alphas, dim=0)
    alpha_bar_t = alphas_bar[t]

    # final marginal at time t
    mu_t = torch.sqrt(alpha_bar_t) * mu0
    Sigma_t = (1 - alpha_bar_t) * torch.eye(m0['d_out'], device=x0.device) + alpha_bar_t * Sigma0

    return mu_t, Sigma_t


In [18]:
# Suppose we already have:
# marginals_t0 = precompute_t0_marginals_sparse_full(...)
T0 = 300
T1 = 200
T2 = 100
betas = torch.linspace(1e-4,0.02,steps=T0+T1+T2)
betas_per_level = [
    betas[0:T0],
    betas[T0:T1+T0],
    betas[T1+T0:],
]
betas_per_level[1]

tensor([0.0101, 0.0101, 0.0101, 0.0102, 0.0102, 0.0102, 0.0103, 0.0103, 0.0103,
        0.0104, 0.0104, 0.0104, 0.0105, 0.0105, 0.0105, 0.0106, 0.0106, 0.0106,
        0.0107, 0.0107, 0.0107, 0.0108, 0.0108, 0.0108, 0.0109, 0.0109, 0.0109,
        0.0110, 0.0110, 0.0110, 0.0111, 0.0111, 0.0111, 0.0112, 0.0112, 0.0112,
        0.0113, 0.0113, 0.0113, 0.0114, 0.0114, 0.0114, 0.0115, 0.0115, 0.0115,
        0.0116, 0.0116, 0.0116, 0.0117, 0.0117, 0.0117, 0.0118, 0.0118, 0.0118,
        0.0119, 0.0119, 0.0119, 0.0120, 0.0120, 0.0120, 0.0121, 0.0121, 0.0121,
        0.0122, 0.0122, 0.0122, 0.0123, 0.0123, 0.0123, 0.0124, 0.0124, 0.0124,
        0.0125, 0.0125, 0.0125, 0.0126, 0.0126, 0.0126, 0.0127, 0.0127, 0.0127,
        0.0128, 0.0128, 0.0128, 0.0129, 0.0129, 0.0129, 0.0130, 0.0130, 0.0130,
        0.0131, 0.0131, 0.0131, 0.0132, 0.0132, 0.0132, 0.0133, 0.0133, 0.0133,
        0.0134, 0.0134, 0.0134, 0.0135, 0.0135, 0.0135, 0.0136, 0.0136, 0.0136,
        0.0137, 0.0137, 0.0137, 0.0138, 

In [19]:
# Suppose we already have:
level_shapes = [(1,8,8), (1,4,4), (1,2,2)]
downsamplers = [AvgPoolDownsample(2), AvgPoolDownsample(2)]
downsample_sigmas = [None, None]  # correctly handle variance scaling

marginals_t0 = precompute_t0_marginals_sparse_full(
    downsamplers, 
    downsample_sigmas,
    level_shapes, 
    betas_per_level    
)

# Level 0
print("Level 0:", marginals_t0[0]['mu0'].shape, marginals_t0[0]['Sigma0'].shape)
# Level 1
print("Level 1:", marginals_t0[1]['mu0'].shape, marginals_t0[1]['Sigma0'].shape)
# Level 2
print("Level 2:", marginals_t0[2]['mu0'].shape, marginals_t0[2]['Sigma0'].shape)


x0_flat = torch.randn((64,1))  # flattened level 0

# Get marginal at level 1, t=3
mu_t, Sigma_t = get_marginal(l=1, t=100, x0=x0_flat,
                             marginals_t0=marginals_t0,
                             betas_per_level=betas_per_level)

print("mu_t shape:", mu_t.shape)
print("Sigma_t shape:", Sigma_t.shape)

# Get marginal at level 2, t=1
mu_t2, Sigma_t2 = get_marginal(l=2, t=50, x0=x0_flat,
                               marginals_t0=marginals_t0,
                               betas_per_level=betas_per_level)

print("mu_t2 shape:", mu_t2.shape)
print("Sigma_t2 shape:", Sigma_t2.shape)

mu_t, Sigma_t, mu_t2, Sigma_t2

Level 0: torch.Size([64, 64]) torch.Size([64, 64])
Level 1: torch.Size([16, 64]) torch.Size([16, 16])
Level 2: torch.Size([4, 64]) torch.Size([4, 4])
mu_t shape: torch.Size([16, 1])
Sigma_t shape: torch.Size([16, 16])
mu_t2 shape: torch.Size([4, 1])
Sigma_t2 shape: torch.Size([4, 4])


(tensor([[-0.2565],
         [ 0.4394],
         [ 0.0226],
         [ 0.0798],
         [-0.1536],
         [-0.3669],
         [ 0.2603],
         [ 0.1496],
         [-0.0044],
         [ 0.0886],
         [-0.2079],
         [ 0.4561],
         [-0.6858],
         [ 0.1893],
         [ 0.1255],
         [ 0.0532]]),
 tensor([[0.9339, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.9339, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.9339, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.9339, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.9339, 0.0000, 0.0000, 0.0000, 0.0000,
